In [ ]:
# default_exp config

from nbdev.showdoc import show_doc
from utilities.ipynb_docgen import *

# Configuration data and basic functions
> Basic functions and configuration stuff

Implements:

- Cache
- Config
- MJD
- UTC, UTCnow

In [ ]:
#export
import os, sys, warnings, pickle
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import numpy as np


In [ ]:
#export

class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object <br> This is the folder where the index and data files are saved.
    - `clear` -- set True to clear the cache on initialization

    This uses pickle to save objects, associated with a hashable key, which is used to index the
    filename in a file `index.pkl` in the same folder.

    The `__call__` function is a convenient way to use it, so one call may either store a new entry or retrieve an existing one.

    """

    def __init__(self, path, clear:bool=False):

        self.path = Path(path) if path else None
        if self.path is None: return
        if not self.path.exists() :
            print(f'Warning: cache Path {self.path} does not exist, cache disabled ',file=sys.stderr)
            self.path=None
            return

        self.index_file = self.path/'index.pkl'

        if self.path.exists():
            if clear:
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)

    def _load_index(self):
        if not self.index_file.exists():
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))

    def add(self, key, object,  exist_ok=False):
        if not self.path: return
        assert type(key)==str, f'Expect key to be a string, got {key}'
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index()

        with open(self.path/filename, 'wb') as file:
            pickle.dump(object, file )


    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        # temp fix
        if str(filename)[0]=='/': filename=self.path/filename.name
        
        if not (self.path/filename).exists():
            # perhaps deleted by another instance?
            print(f'File for Cache key {key} not found, removing entry', file='sys.stderr')
            selt.pop(key)
            return None
        with open(self.path/filename, 'rb') as file:
            ret = pickle.load(file)
        return ret

    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file:
                f.unlink()
        super().clear()

        self._dump_index()

    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self.path/self[key]
        try:
            filename.unlink()
        except:
            print(f'Failed to unlink file {filename}', file=sys.stderr)
            raise
        super().pop(key)
        self._dump_index()


    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use

        - `key` -- key to use, usually a string. Must be hashable <br>
            If None, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed
        - `overwrite` -- if set, overwrite previous entry if exists

        Example:
        <pre>
        mycache = Cache('/tmp/thecache', clear=True)

        def myfun(x):
            return x

        result = mycache('mykey', myfun, x=99,  description='My data')

        </pre>

        """

        if key is None or self.path is None:
            return func(*pars, **kwargs)


        if description:
            print(f'{description}: {"Saving to" if key not in self or overwrite else "Restoring from"} cache', end='')
            print('' if key == description else f' with key "{key}"')
        ret = self.get(key)
        if ret is None or overwrite:
            ret = func(*pars, **kwargs)
            self.add(key, ret, exist_ok=overwrite)
        return ret

    def show(self, starts_with=''):
        import datetime
        if not self.path: return 'Cache not enabled'
        if len(self.items())==0: return f'Cache at {self.path} is empty\n'
        title = 'Cache contents' if not starts_with else f'Cache entries starting with {starts_with}'
        s = f'{title}\n {"key":30}   {"size":>10}  {"time":20} {"name"}, folder {self.path}\n'
        for name, value in self.items():
            # temporary: override file's path if set 
            if str(value)[0]=='/':
                value = self.path/value.name
            if name is None or not name.startswith(starts_with) : continue
            try:
                stat = (self.path/value).stat()
                size = stat.st_size
                mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
                s += f'  {name:30s}  {size:10}  {mtime:20} {value}\n'
            except Exception as msg:
                s += f'{name} -- file not found\n'
                raise
        return s

    def __str__(self):
        return self.show()

In [ ]:
show_doc(Cache.__call__)

<h4 id="Cache.__call__" class="doc_header"><code>Cache.__call__</code><a href="__main__.py#L103" class="source_link" style="float:right">[source]</a></h4>

> <code>Cache.__call__</code>(**`key`**, **`func`**, **\*`pars`**, **`description`**=*`''`*, **`overwrite`**=*`False`*, **\*\*`kwargs`**)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If None, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed
- `overwrite` -- if set, overwrite previous entry if exists

Example:
<pre>
mycache = Cache('/tmp/thecache', clear=True)

def myfun(x):
    return x

result = mycache('mykey', myfun, x=99,  description='My data')

</pre>

### Cache test

In [ ]:
#collapse_hide
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
    print(f'After remove:\n{c}')
    c.clear()

test_path = Path('/tmp/cache_test')
test_path.mkdir(exist_ok=True)
cache_test(test_path)


Clearing cache!
Test: Saving to cache with key "four"
Test: Restoring from cache with key "four"
Before remove:
Cache contents
 key                                    size  time                 name, folder /tmp/cache_test
  one                                     18  2022-07-16 04:47     cache_file_a914b484ec77559.pkl
  two                                     18  2022-07-16 04:47     cache_file_252f4e04c4c9e39.pkl
  four                                    23  2022-07-16 04:47     cache_file_6306a9b495235f4.pkl

After remove:
Cache contents
 key                                    size  time                 name, folder /tmp/cache_test
  one                                     18  2022-07-16 04:47     cache_file_a914b484ec77559.pkl
  two                                     18  2022-07-16 04:47     cache_file_252f4e04c4c9e39.pkl



In [ ]:
# export
class Config():
    defaults=\
    """
        verbose         : 1 # set to zero for no output
        warnings        : ignore # 

        datapath        : ~/wtlike-data # where to find data--must be set
        cachepath       : ~/.cache/wtlike #

        # Expect 4FGL FITS file, e.g.,  gll_psc_v28.fit
        catalog_file    : 
        
        # multiprocessing
        pool_size       : 1 # number of pool processes to use

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning -- actually determined by weight run
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024
        nest            : True

        # data selection for cell creation
        week_range      : []  # default all weeks found
        time_bins       : [0, 0, 7] # full MJD range, 7-day cells
        exp_min         : 5    # threshold for exposure per day, in cm^2 Ms units.

        # cell fitting
        use_kerr        : True  # Use the Kerr power-law exposure weighting
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    """

    def __init__(self, **kwargs):
        import yaml, warnings
        from yaml import SafeLoader

        # parameters: first defaults, then from ~/.config/wtlike/config.yaml, then kwars
        pars = yaml.load(self.defaults, Loader=SafeLoader)
        dp = Path('~/.config/wtlike/config.yaml').expanduser()
        if dp.is_file():
            with open(dp, 'r') as inp:
                userpars = yaml.load(inp, Loader=SafeLoader)
            pars.update(userpars)
            #print(f'update from user file {dp}: {userpars}')
        pars.update(kwargs)

        self.__dict__.update(pars)

        # set warnings filter if requested
        if self.warnings is not None and self.warnings != 'None':
            #print(f'*** set warnings filter to "{self.warnings}" ***')
            warnings.simplefilter(self.warnings)

        self.energy_edges = ee=np.logspace(*self.energy_edge_pars)
        self.energy_bins = np.sqrt(ee[1:] * ee[:-1])
        if not self.week_range:
            self.week_range = (None, None)

       # set up, check files paths
        self.error_msg=''
        if self.datapath is None:
            self.error_msg+='\ndatapath must be set to a folder with wtlike data'
            
        else:
            self.datapath = df = Path(self.datapath).expanduser()
            if not (self.datapath.is_dir() or  self.datapath.is_symlink()):
                self.error_msg+=f'\ndata_folder "{df}" is not a directory or symlink'
            subs = 'aeff_files weight_files data_files'.split()
            if self.error_msg=='':
                for sub in subs:
                    if not ( (df/sub).is_dir() or  (df/sub).is_symlink()) :
                        self.error_msg+=f'\n{df/sub} is not a directory or symlink'

        self.cachepath =  Path(self.cachepath).expanduser()
        os.makedirs(self.cachepath, exist_ok=True)
        if not self.cachepath.is_dir():
            self.error_msg +=f'cachepath {self.cachepath} is not a folder.'
            
        # look for 4FGL catalog file, gll_psc_v28.fit currently
        fail = False
        if self.catalog_file is None or self.datapath is None:
            t = Path(self.datapath).expanduser()
            u = sorted(list(t.glob('gll_psc_v*.fit')))
            if len(u)>0:
                self.catalog_file = u[-1]
            else: 
                fail = True
        elif Path(self.catalog_file).expanduser().is_file():
            self.catalog_file = Path(config.catalog_file).expanduser()
        else: fail=True
        
        if fail:
            warnings.warn('There is no link to 4FGL catalog file: set "catalog_file" in your config.yaml'
                  ' or specify if in the Config() call')


    @property
    def cache(self):
        if not hasattr(self, '_cache'):
            self._cache = Cache(self.cachepath, clear=False)
        return self._cache

    @property
    def valid(self):
        if len(self.error_msg)==0: return True
        print(f'wtlike configuration is invalid:\n{self.error_msg}',file=sys.stderr)
        return False

    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files' or name.startswith('_'): continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)
    def get(self, *pars): return self.__dict__.get(*pars)

In [ ]:
show_doc(Config)

<h2 id="Config" class="doc_header"><code>class</code> <code>Config</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Config</code>(**\*\*`kwargs`**)



In [ ]:
#collapse-hide
@ipynb_doc
def config_summary():
    """
    
    #### config.Config -- parameters are from three sources:
    - defaults
    - the file `~/.config/wtlike/config.yaml` if it exists
    - keyword args in Config constructor. For example, to suppress all printout:
        ```
    config = Config(verbose=0)
    ```
    
    ##### Config defaults
    This is yaml-format, corresponding to `config.yaml`.
    {config_defaults}
    
    
    #### Config contents as set up here
    {config_text}   


    
    ##### config.cache -- a file cache
    The class `Cache`, available from `config.cache` implements a file cache in the folder
    `{config.cachepath}`
    
    {cache_text}
    """
    config = Config()
    config_defaults = Config.defaults
    config_text = monospace(config, summary='config parameter list')

    try:
        cache_text = monospace(config.cache, 'current cache contents' )
    except Exception as msg:
        cache_text = f'(Failed: {msg})'


    return locals()
config_summary() 


#### config.Config -- parameters are from three sources:
- defaults
- the file `~/.config/wtlike/config.yaml` if it exists
- keyword args in Config constructor. For example, to suppress all printout:
    ```
config = Config(verbose=0)
```

##### Config defaults
This is yaml-format, corresponding to `config.yaml`.

        verbose         : 1 # set to zero for no output
        warnings        : ignore # 

        datapath        : ~/wtlike-data # where to find data--must be set
        cachepath       : ~/.cache/wtlike #

        # Expect 4FGL FITS file, e.g.,  gll_psc_v28.fit
        catalog_file    : 
        
        # multiprocessing
        pool_size       : 1 # number of pool processes to use

        # data cuts, processing
        radius          : 4
        cos_theta_max   : 0.4
        z_max           : 100
        offset_size     : 2.e-06  # scale factor used for event time

        # binning -- actually determined by weight run
        energy_edge_pars : [2,6,17] # pars for np.logspace
        etypes          : [0, 1] # front, back
        nside           : 1024
        nest            : True

        # data selection for cell creation
        week_range      : []  # default all weeks found
        time_bins       : [0, 0, 7] # full MJD range, 7-day cells
        exp_min         : 5    # threshold for exposure per day, in cm^2 Ms units.

        # cell fitting
        use_kerr        : True  # Use the Kerr power-law exposure weighting
        likelihood_rep  : poisson
        poisson_tolerance : 0.2

    


#### Config contents as set up here
<details  class="nbdoc-description" >  <summary> config parameter list </summary>  <div style="margin-left: 5%;"><pre>Configuration parameters <br>  verbose         : 1<br>  warnings        : None<br>  datapath        : /mnt/c/Users/thbur/onedrive/fermi/wtlike-data<br>  cachepath       : /home/burnett/.cache/wtlike<br>  catalog_file    : /mnt/c/Users/thbur/onedrive/fermi/wtlike-data/gll_psc_v28.fit<br>  pool_size       : 4<br>  radius          : 4<br>  cos_theta_max   : 0.4<br>  z_max           : 100<br>  offset_size     : 2e-06<br>  energy_edge_pars : [2, 6, 17]<br>  etypes          : [0, 1]<br>  nside           : 1024<br>  nest            : True<br>  week_range      : (None, None)<br>  time_bins       : [0, 0, 7]<br>  exp_min         : 5<br>  use_kerr        : True<br>  likelihood_rep  : poisson<br>  poisson_tolerance : 0.2<br>  keep_pixels     : True<br>  energy_edges    : [1.00000000e+02 1.77827941e+02 3.16227766e+02 5.62341325e+02<br> 1.00000000e+03 1.77827941e+03 3.16227766e+03 5.62341325e+03<br> 1.00000000e+04 1.77827941e+04 3.16227766e+04 5.62341325e+04<br> 1.00000000e+05 1.77827941e+05 3.16227766e+05 5.62341325e+05<br> 1.00000000e+06]<br>  energy_bins     : [1.33352143e+02 2.37137371e+02 4.21696503e+02 7.49894209e+02<br> 1.33352143e+03 2.37137371e+03 4.21696503e+03 7.49894209e+03<br> 1.33352143e+04 2.37137371e+04 4.21696503e+04 7.49894209e+04<br> 1.33352143e+05 2.37137371e+05 4.21696503e+05 7.49894209e+05]<br>  error_msg       : <br></pre></div> </details>   



##### config.cache -- a file cache
The class `Cache`, available from `config.cache` implements a file cache in the folder
`/home/burnett/.cache/wtlike`

<details  class="nbdoc-description" >  <summary> current cache contents </summary>  <div style="margin-left: 5%;"><pre>Cache contents<br> key                                    size  time                 name, folder /home/burnett/.cache/wtlike<br>  PSR J0835-4510_data              156120367  2022-06-06 12:55     cache_file_ae6854be273acdc.pkl<br>  PSR J0633+1746_data              115577595  2022-06-06 13:00     cache_file_5707380cef0a5b.pkl<br>  PSR J1709-4429_data              120303131  2022-06-06 13:05     cache_file_7500f4770cd889.pkl<br>  PSR J2021+4026_data              130940729  2022-06-06 13:11     cache_file_a8dc38f04aab018.pkl<br>  PSR J1836+5925_data              125537569  2022-06-06 13:17     cache_file_a2b8a18b80eb164.pkl<br>  PSR J2021+3651_data              126539471  2022-06-06 13:23     cache_file_6661649ffbd07a9e.pkl<br>  PSR J0007+7303_data              134394303  2022-07-04 09:49     cache_file_cfa096edb809c60.pkl<br>  PSR J1809-2332_data              116347883  2022-06-06 13:40     cache_file_fb3fdcb2262565.pkl<br>  PSR J1826-1256_data              122240951  2022-06-06 13:45     cache_file_335ec043a4024484.pkl<br>  PSR J1907+0602_data              118468339  2022-06-06 13:51     cache_file_55b744805b5c9ff9.pkl<br>  PSR J1418-6058_data              132775053  2022-06-06 13:58     cache_file_29d5c1a8500b93e8.pkl<br>  PSR J1057-5226_data              101444077  2022-06-06 14:03     cache_file_384e41fd4ebdac81.pkl<br>  PSR J1813-1246_data              117386407  2022-06-06 14:08     cache_file_08c2148395b6f1f.pkl<br>  PSR J1028-5819_data              122583649  2022-06-06 14:15     cache_file_812ad9ec583a961.pkl<br>  PSR J2229+6114_data              133820383  2022-06-06 14:21     cache_file_e476bf55a06fa3e.pkl<br>  PSR J1048-5832_data              120018647  2022-06-06 14:27     cache_file_70b0714d172f3f2.pkl<br>  PSR J1732-3131_data              123636019  2022-06-06 14:33     cache_file_19edcba9cfff048.pkl<br>  PSR J1413-6205_data              132165683  2022-06-06 14:40     cache_file_6b3b60386724b1db.pkl<br>  PSR J1747-2958_data              126607289  2022-06-06 14:46     cache_file_2f67d22cb93688bb.pkl<br>  PSR J1952+3252_data              114920081  2022-06-06 14:52     cache_file_512aa5183afaad0.pkl<br>  P88Y3243_data                     88630555  2022-06-07 05:16     cache_file_66a9d9663efb97.pkl<br>  P88Y5020_data                    119817913  2022-06-07 10:33     cache_file_29e7a2e2944fb978.pkl<br>  504H-0317_data                   130475249  2022-06-07 10:39     cache_file_cb1897be30807d5.pkl<br>  P88Y6266_data                     99892151  2022-06-13 17:57     cache_file_35907c8f78cbed7c.pkl<br>  P88Y2172_data                    117721945  2022-06-17 11:49     cache_file_175f18c1a956e4d.pkl<br>  P88Y0096_data                    133548253  2022-07-04 09:54     cache_file_3c73e1e34f4c55.pkl<br>  P88Y0521_data                     95747295  2022-07-02 09:46     cache_file_7f6ab463d3ea18fd.pkl<br>  P88Y4060_data                     99826151  2022-07-02 10:05     cache_file_43042fb37426e122.pkl<br>  P88Y1134_data                     94193191  2022-07-02 09:51     cache_file_24a8a2282c30fdea.pkl<br>  P88Y6057_data                     89569443  2022-07-02 10:10     cache_file_4c75eeb98fde1be8.pkl<br>  P88Y1710_data                    125661037  2022-07-02 09:56     cache_file_342f3a94f9ee1f36.pkl<br>  P88Y1932_data                     87756585  2022-07-02 10:00     cache_file_5d597895910d1291.pkl<br>  P88Y0643_data                     97581865  2022-07-03 08:42     cache_file_4f9f229b3e50c0d6.pkl<br>  P88Y0514_data                     96424943  2022-07-03 13:01     cache_file_49fb71b2c22b562d.pkl<br>  PSR J0633+1746_weeks_9-11           419065  2022-07-04 10:03     cache_file_dd5ac11dbd01293.pkl<br>  P88Y3157_weeks_9-165              20704939  2022-07-04 10:06     cache_file_cd697fdc73ce09.pkl<br>  PSR J0534+2200_data              111891877  2022-07-13 01:08     cache_file_737398512a45c22.pkl<br>  P88Y3157_data                     88180753  2022-07-13 01:15     cache_file_5f7b91e889cf5dcf.pkl<br>  P88Y0615_data                     94004743  2022-07-14 13:03     cache_file_19435ee6bf1a46a2.pkl<br>  PSR J2021+4026-nk                138951485  2022-07-15 09:24     cache_file_6e0c2e81f58fab52.pkl<br></pre></div> </details>


## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time
- UTCnow: current ISO time

In [ ]:
#export

day = 24*3600.
first_data=54683
#mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
# From a FT2 file header
# MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
# MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
mission_start = 51910.00074287037
from datetime import datetime
from astropy.time import Time

def MJD(arg):
    """ convert MET or UTC to MJD
    """

    if type(arg)==str:
        if arg=='now':
            return Time(datetime.utcnow()).mjd
        while len(arg.split('-'))<3:
            arg+='-1'
        return Time(arg, format='iso').mjd
    return (mission_start + arg/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

def UTCnow():
    """ current UTC """
    t=datetime.utcnow()
    return f'UTC {t.year}-{t.month:02d}-{t.day} {t.hour:02d}:{t.minute:02d}'

def mission_week(mjd):
    """ return the mission week number for a MJD value
    (Note that week #0 starts on UTC Thursday 2008-05-29 00:00, the first data is in week 9, and that week 525 is missing)
    """
    return (mjd-54615)//(7)

def mjd_range(start,stop, make_round=True):
    """Return a tuple with a valid MJD range
    If either is greater than first_data, interpret as MJD
    otherwise offset from first_data, or now
    
    So 0,0 is full range, -7, 0 is last 7 days, 0,7 is first 7 days

    """
    a,b = start, stop
    # extract MJD range
    now = MJD('now')
    if a<0: a = now-a
    elif a < first_data : a+=first_data 
    if b <= 0 :  b = now-b
    elif b < first_data: b += first_data
    
    return (round(a),round(b)) if make_round else (a,b)  

In [ ]:
UTC(MJD(0)), UTC(first_data), UTCnow(), MJD('now'), UTC(MJD('now')), mjd_range(0,0)

('2001-01-01 00:01',
 '2008-08-05 00:00',
 'UTC 2022-07-16 11:47',
 59776.49153800692,
 '2022-07-16 11:47',
 (54683, 59776))

In [ ]:
np.arange(*[mission_week(x) for x in mjd_range(0,100)])

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'
assert MJD('2008')==54466

In [ ]:
# export
class FermiInterval():
    """For iteration thru (start,stop) tuples in the Fermi data set
    """
    def __init__(self, interval=30, offset=0):
        from wtlike.config import  MJD, first_data
        self.interval=interval
        a,b = first_data+offset, MJD('now')
        self.mm = np.arange(a,round(b),interval)
        
    def __len__(self):
        return len(self.mm)-1
    
    def __getitem__(self, k):
        return (self.mm[k], self.mm[k+1],) if k>=0 else (self.mm[k-1],self.mm[k])
    
    def __call__(self, k):
        """ Return a timeinterval tuple
        """
        a,b = self[k]
        return (a,b,self.interval)
    
class FermiMonth(FermiInterval):
    def __init__(self):
        super().__init__(interval=30)
        
class FermiWeek(FermiInterval):
    def __init__(self):
        super().__init__(interval=7)

### Miscellaneous utilities
- bins_size_name
- decorate_with
- Timer

In [ ]:
#export
def bin_size_name(bins):
    """Provide a nice name, e.g., 'day' for a time interval
    """
    if np.isscalar(bins) :
        binsize = bins
    else:
        binsize = np.mean(bins)

    def check_unit(x):
        unit_table = dict(year=1/365.25, week=1/7, day=1, hour=24, min=24*60) #, min=24*60, s=24*3600)
        for name, unit in unit_table.items():
            t = x*unit
            r = np.mod(t+1e-9,1)

            if r<1e-6 : #or t>1:
                return t, name
        return x, 'day'
    n, unit =  check_unit(binsize)
    nt = f'{n:.0f}' if np.mod(n,1)<1e-3 else f'{n:.1f}'
    return f'{nt}-{unit}'# if n>1 else f'{unit}'

In [ ]:
print([bin_size_name(x) for x in (1/96, 1,2,7, 14, 365.25)])

['15-min', '1-day', '2-day', '1-week', '2-week', '1-year']


In [ ]:
# export
def decorate_with(other_func):
    def decorator(func):
        func.__doc__ += other_func.__doc__
        return func
    return decorator

In [ ]:
# export
import time

class Timer():
    """Usage:
    ```
    with Timer() as t:
        time.sleep(5)
    print(t)
    ```
    """

    def __init__(self):
        self.t=time.time()
        self.exit_time=1e6

    def __enter__(self):
        return self
    def __exit__(self, *pars):
        self.exit_time = time.time()-self.t
    def __repr__(self):
         return  f'elapsed time: {self.elapsed:.1f}s ({self.elapsed/60:.1f} min)'
    @property
    def elapsed(self):
        return min(time.time()-self.t, self.exit_time)

In [ ]:
# collapse-show
with Timer() as t:

    time.sleep(2)
    print('intemediate',t)
    time.sleep(3)
print('Final',t)
assert(abs(t.elapsed -5)<0.1), f'wrong elapsed time: {t.elapsed}'


intemediate elapsed time: 2.0s (0.0 min)
Final elapsed time: 5.0s (0.1 min)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_data_man.ipynb.
Converted 02_effective_area.ipynb.
Converted 03_exposure.ipynb.
Converted 03_sources.ipynb.
Converted 04_load_data.ipynb.
Converted 04_simulation.ipynb.
Converted 04_skymaps.ipynb.
Converted 05_source_data.ipynb.
Converted 06_poisson.ipynb.
Converted 07_loglike.ipynb.
Converted 08_cell_data.ipynb.
Converted 09_lightcurve.ipynb.
Converted 10-time_series.ipynb.
Converted 14_bayesian.ipynb.
Converted 90_main.ipynb.
Converted 99_presentation.ipynb.
Converted 99_tutorial.ipynb.
Converted index.ipynb.
Sat Jul 16 04:47:59 PDT 2022
